# <font color='yellow'>PA005: High Value Customer Identification (Insiders)

# Basic setups

## Imports

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import umap.umap_ as umap
# pip install umap-learn  to work
from IPython.core.display     import HTML
from matplotlib import pyplot as plt
from pandas_profiling import ProfileReport 
from sklearn import cluster       as c
from sklearn import metrics       as m
from sklearn import ensemble      as en
from sklearn import preprocessing as pp
from sklearn import decomposition as dd
from sklearn import manifold      as mn
from sklearn import mixture       as mx
from sklearn.manifold import TSNE
from sklearn.neighbors import NearestNeighbors
from scipy.cluster import hierarchy as hc

from plotly import express as px

from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer

## Helper Functions

In [2]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    sns.set()
jupyter_settings()

Populating the interactive namespace from numpy and matplotlib


In [3]:
# Supress Scientific Notation
np.set_printoptions(suppress=True)
pd.set_option('display.float_format', '{:.2f}'.format)

# Data description

## Data Load

In [4]:
df_raw = pd.read_csv(r'../data/raw/Ecommerce.csv')

## Data Overview

In [5]:
df_raw.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Unnamed: 8
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,17850.00,United Kingdom,NaN
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,17850.00,United Kingdom,NaN
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,17850.00,United Kingdom,NaN
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,17850.00,United Kingdom,NaN
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,17850.00,United Kingdom,NaN


In [6]:
print('The original dataset has {} transations registers with {} attributes.'.format(df_raw.shape[0], df_raw.shape[1]) )

The original dataset has 541909 transations registers with 9 attributes.


In [7]:
df_raw.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country', 'Unnamed: 8'],
      dtype='object')

Tasks:

    1) Change the name columns to snacke case
    
    2) Check "Unnamed: 8" column

## Check NaN

In [8]:
df_raw.isna().sum() / len(df_raw) *100

InvoiceNo       0.00
StockCode       0.00
Description     0.27
Quantity        0.00
InvoiceDate     0.00
UnitPrice       0.00
CustomerID     24.93
Country         0.00
Unnamed: 8    100.00
dtype: float64

Tasks:

4) Check "Description" and "CustomerID" NaN

In [9]:
print('''There are a considerable number of transations without customers. Identify the customers is the main objective of this project, 
so this is a big problem. Through this cycle. we have a lot of problems with the non identified customers, so we decided to drop then in this 
first cycle of the project. Maybe in the next cycle somethin different can be done. After all, we discovery that dropping all the clientes without id, 
we drop all the "descriptions" NaN as well''')

There are a considerable number of transations without customers. Identify the customers is the main objective of this project, 
so this is a big problem. Through this cycle. we have a lot of problems with the non identified customers, so we decided to drop then in this 
first cycle of the project. Maybe in the next cycle somethin different can be done. After all, we discovery that dropping all the clientes without id, 
we drop all the "descriptions" NaN as well


## Data types

In [10]:
df_raw.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
Unnamed: 8     float64
dtype: object

Tasks:

5) Change InvoiceNo type to int, change InvoiceDate type to datetime and hange CustomerID type  to int.

## Check duplicated

In [11]:
df_raw['CustomerID'].duplicated().sum()

537536

In [12]:
print('From a business perspective, none of the repeated attributes will be a problem unless there are more than one record with all inputs equals.')

From a business perspective, none of the repeated attributes will be a problem unless there are more than one record with all inputs equals.


Task
8) Check if there are duplicated registers

## Check attributes

### Attribute 'InvoiceNo'

In [13]:
df_raw['InvoiceNo'].unique()

array(['536365', '536366', '536367', ..., '581585', '581586', '581587'],
      dtype=object)

Task: 

9) Check the invoices with letters. They seem to be devolutions

### Attributes 'Description'

In [14]:
df_raw['Description'].unique()

array(['WHITE HANGING HEART T-LIGHT HOLDER', 'WHITE METAL LANTERN',
       'CREAM CUPID HEARTS COAT HANGER', ..., 'lost',
       'CREAM HANGING HEART T-LIGHT HOLDER',
       'PAPER CRAFT , LITTLE BIRDIE'], dtype=object)

Task: 10)Check the strange names

## Statistics data overview

In [15]:
df_raw.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,541909.00,9.55,218.08,-80995.00,1.00,3.00,10.00,80995.00
UnitPrice,541909.00,4.61,96.76,-11062.06,1.25,2.08,4.13,38970.00
CustomerID,406829.00,15287.69,1713.60,12346.00,13953.00,15152.00,16791.00,18287.00
Unnamed: 8,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Task 9) Check negative unit prices and quantities

In [ ]:
#stock code

#df2 = df2[~df2['stock_code'].isin (['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B', 'CRUK'])]

# <font color='green'> Data Processing

In [165]:
df3 = df_raw.copy()

## Rename columns

In [166]:
df3.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country', 'Unnamed: 8'],
      dtype='object')

In [167]:
cols_new = ['invoice_no', 'stock_code', 'description','quantity', 'invoice_date','unit_price', 'customer_id', 'country', 'unnamed']

In [168]:
df3.columns = cols_new #change to snacke case

## Check "Unnamed: 8" attribute

In [169]:
print('All the Unnamed are NaN. There is no relevante information in this feature. It will be dropped.')

All the Unnamed are NaN. There is no relevante information in this feature. It will be dropped.


In [170]:
df3 = df3.drop(columns=['unnamed'], axis=1) 

## Solving NaN

### Attribute 'customer_id

In [171]:
df3.isna().sum()

invoice_no           0
stock_code           0
description       1454
quantity             0
invoice_date         0
unit_price           0
customer_id     135080
country              0
dtype: int64

In [172]:
df3 = df3.dropna(subset = ['customer_id'], axis=0)

In [173]:
df3.isna().sum()

invoice_no      0
stock_code      0
description     0
quantity        0
invoice_date    0
unit_price      0
customer_id     0
country         0
dtype: int64

### Attribute 'description'

In [174]:
print('The "description" NaN were dropped together with the "customer_id" NaN')

The "description" NaN were dropped together with the "customer_id" NaN


## Change dtypes

### Attribute 'invoice_number'

In [175]:
df3['invoice_no'].unique()

array(['536365', '536366', '536367', ..., '581585', '581586', '581587'],
      dtype=object)

In [176]:
#df3['invoice_no'] = df3['invoice_no'].astype(int64)
print('According with the code error, there are invoices with number with letters. For now it will be keep as a object')

According with the code error, there are invoices with number with letters. For now it will be keep as a object


### Attribute 'invoice_date'

In [177]:
df3['invoice_date'] = pd.to_datetime(df3['invoice_date'], format = '%d-%b-%y')
# use %b because the month is in a string in the df

In [178]:
df3.dtypes

invoice_no              object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
unit_price             float64
customer_id            float64
country                 object
dtype: object

### Attribute 'customer_id'

In [179]:
df3['customer_id'] = df3['customer_id'].astype(int64)

In [180]:
df3.dtypes

invoice_no              object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
unit_price             float64
customer_id              int64
country                 object
dtype: object

## Negative values

From here, we have to take a direction. The negative values problably are from returns. We can just exclude them, exclude the in an out or we can create features for returns. From this point, we decided to sum all the values for customer, and those who ended up with a negative or zero amount will be dropped. We can divided the dataframe in two, one with the purchases, one just with the returns, but it will be done in the next cycle.

In [181]:
df3['subtotal'] = df3['unit_price'] * df3['quantity'] 

In [183]:
bad_customers = df3[['customer_id','subtotal']].groupby('customer_id').sum().reset_index()

In [187]:
bad_customers.head(20)

,customer_id,subtotal
0,12346,0.00
89,12454,-0.00
125,12503,-1126.00
127,12505,-4.50
212,12605,-7.50
264,12666,-227.44
415,12870,-14.90
467,12943,-3.75
619,13154,-611.86
692,13256,0.00


In [185]:
bad_customers = bad_customers[bad_customers['subtotal'] <=0]
bad_list = bad_customers['customer_id'].to_list() # list with the "bad" customers ids

In [189]:
df3 = df3[~df3['customer_id'].isin(bad_list)]

## Solve Duplicates

In [192]:
#next cycle

## Check 'Description' inputs

In [193]:
#quem são os invoices que contém letras?
df_letter_invoices = df3.loc[df3['invoice_no'].apply (lambda x: bool(re.search ( '[^0-9]+', x))), :] 
print('Total number of invoices with letters: {}'.format (len (df_letter_invoices)))

print('Total number of invoices with negative quantity: {}'.format( len(df_letter_invoices['quantity'] <0)))

Total number of invoices with letters: 8706
Total number of invoices with negative quantity: 8706


Along the work, we note some descriptions of sales that seem not be sensible
D, DOT. M. S., AMazon fee, pads, b, cruk

# <font color='orange'> Feature Engineering

In [194]:
df4 = df3.copy()

## Data Filtering

In [195]:
df4.drop(columns = ['stock_code', 'description','country'], axis=1, inplace=True)

In [196]:
df4.head(10)

,invoice_no,quantity,invoice_date,unit_price,customer_id,subtotal
0,536365,6,2016-11-29,2.55,17850,15.30
1,536365,6,2016-11-29,3.39,17850,20.34
2,536365,8,2016-11-29,2.75,17850,22.00
3,536365,6,2016-11-29,3.39,17850,20.34
4,536365,6,2016-11-29,3.39,17850,20.34
5,536365,2,2016-11-29,7.65,17850,15.30
6,536365,6,2016-11-29,4.25,17850,25.50
7,536366,6,2016-11-29,1.85,17850,11.10
8,536366,6,2016-11-29,1.85,17850,11.10
9,536367,32,2016-11-29,1.69,13047,54.08


## Average Ticket Price

In [197]:
basket_size = df4[['invoice_no','subtotal']].groupby('invoice_no').sum().reset_index()

In [198]:
basket_size.rename(columns = {'subtotal':'avg_ticket'}, inplace = True)            
basket_size.head(10)

,invoice_no,avg_ticket
0,536365,139.12
1,536366,22.20
2,536367,278.73
3,536368,70.05
4,536369,17.85
5,536370,855.86
6,536371,204.00
7,536372,22.20
8,536373,259.86
9,536374,350.40


In [199]:
df4 = pd.merge(df4,basket_size, on ='invoice_no', how = 'left')

In [219]:
df4.sample(15)

,customer_id,avg_ticket,recency,gross_revenue,frequency
389656,13090,355.38,8,8689.39,18
381774,18041,267.40,11,4161.14,22
285693,15756,168.12,61,532.96,4
56012,18159,530.15,301,530.15,1
65243,14004,900.02,43,4582.64,7
311836,18045,189.40,45,189.40,1
255962,13908,907.94,79,2347.05,5
375289,12784,518.42,9,574.42,2
227899,14050,485.98,31,746.36,2
108681,17865,569.20,30,10158.54,27


In [201]:
df4['customer_id'].unique().size

4322

## recency

In [202]:
# day from the last purchase until the last date of the dataset
df_recency = df4[['customer_id','invoice_date']].groupby('customer_id').max().reset_index()
df_recency['recency'] = (df4['invoice_date'].max() - df_recency['invoice_date']).dt.days
df4=pd.merge(df4, df_recency[['customer_id','recency']], on = 'customer_id', how='left')

In [203]:
df4.head()

,invoice_no,quantity,invoice_date,unit_price,customer_id,subtotal,avg_ticket,recency
0,536365,6,2016-11-29,2.55,17850,15.30,139.12,302
1,536365,6,2016-11-29,3.39,17850,20.34,139.12,302
2,536365,8,2016-11-29,2.75,17850,22.00,139.12,302
3,536365,6,2016-11-29,3.39,17850,20.34,139.12,302
4,536365,6,2016-11-29,3.39,17850,20.34,139.12,302


## gross_revenue

In [204]:
df_revenue = df4[['subtotal', 'customer_id']].copy()

In [205]:
df_revenue = df_revenue.groupby('customer_id').sum().reset_index() #total per customer
df_revenue.rename(columns = {'subtotal':'gross_revenue'}, inplace = True)
df_revenue.head()

,customer_id,gross_revenue
0,12347,4310.00
1,12348,1797.24
2,12349,1757.55
3,12350,334.40
4,12352,1545.41


In [206]:
df4 = pd.merge(df4, df_revenue, on='customer_id', how='left')

In [207]:
df4.head()

,invoice_no,quantity,invoice_date,unit_price,customer_id,subtotal,avg_ticket,recency,gross_revenue
0,536365,6,2016-11-29,2.55,17850,15.30,139.12,302,5288.63
1,536365,6,2016-11-29,3.39,17850,20.34,139.12,302,5288.63
2,536365,8,2016-11-29,2.75,17850,22.00,139.12,302,5288.63
3,536365,6,2016-11-29,3.39,17850,20.34,139.12,302,5288.63
4,536365,6,2016-11-29,3.39,17850,20.34,139.12,302,5288.63


## Frequency

In [208]:
df_frequency = df4[['customer_id', 'invoice_no']]

In [209]:
df_frequency = df_frequency.drop_duplicates(subset = 'invoice_no')
df_frequency = df_frequency.groupby('customer_id').count().reset_index() # number of purchases for customer within the dataset

In [210]:
df_frequency.rename(columns={'invoice_no':'frequency'}, inplace = True)

In [211]:
df4 = pd.merge(df4, df_frequency, on='customer_id', how = 'left')

In [212]:
df4.head()

,invoice_no,quantity,invoice_date,unit_price,customer_id,subtotal,avg_ticket,recency,gross_revenue,frequency
0,536365,6,2016-11-29,2.55,17850,15.30,139.12,302,5288.63,35
1,536365,6,2016-11-29,3.39,17850,20.34,139.12,302,5288.63,35
2,536365,8,2016-11-29,2.75,17850,22.00,139.12,302,5288.63,35
3,536365,6,2016-11-29,3.39,17850,20.34,139.12,302,5288.63,35
4,536365,6,2016-11-29,3.39,17850,20.34,139.12,302,5288.63,35


In [213]:
df4.drop(columns = ['quantity','unit_price' ,'subtotal','invoice_date','invoice_no'], axis = 1, inplace = True) # we don't need this features

# Exploratory data analysis (EDA)

In [214]:
df5 = df4.copy()

In [215]:
profile = ProfileReport(df5)
profile.to_file(r'../reports/profile_report.html')

Summarize dataset:   0%|          | 0/19 [00:00<?, ?it/s]

C:\Users\lelos\anaconda3\envs\DSP_002\lib\site-packages\scipy\stats\stats.py:4594: RuntimeWarning: overflow encountered in longlong_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Descriptive Statistics

In [216]:
num_attributes = df5.drop(columns = 'customer_id', axis = 1)

In [217]:
#cental tendency - mean, median
ct1 = pd.DataFrame ( num_attributes.apply (np.mean) ).T
ct2 = pd.DataFrame ( num_attributes.apply (np.median)).T
                    
# dispersion - desvio padrão, mínimo, máximo, range, skew, kurtosis
d1 = pd.DataFrame ( num_attributes.apply (np.std)).T
d2 = pd.DataFrame ( num_attributes.apply (np.min)).T
d3 = pd.DataFrame ( num_attributes.apply (np.max)).T
d4 = pd.DataFrame ( num_attributes.apply (lambda x: x.max() - x.min())).T
d5 = pd.DataFrame ( num_attributes.apply (lambda x: x.skew() )).T
d6 = pd.DataFrame ( num_attributes.apply (lambda x: x.kurtosis() ) ).T

#concatenate

df_metrics = pd.concat ([d2,d3,d4,ct1,ct2,d1,d5,d6]).T.reset_index()
df_metrics.columns = ['attributes', 'min.','max','range', 'mean', 'median', 'std', 'skew', 'kurtosis']
df_metrics

,attributes,min.,max,range,mean,median,std,skew,kurtosis
0,avg_ticket,-168469.60,168469.60,336939.20,724.68,386.31,1491.50,6.73,875.73
1,recency,0.00,373.00,373.00,39.35,15.00,64.49,2.79,8.18
2,gross_revenue,0.00,279489.02,279489.02,11076.01,2618.23,30154.64,5.71,39.86
3,frequency,1.00,248.00,247.00,23.24,8.00,45.51,3.54,12.47


In [153]:
num_attributes.describe()

,avg_ticket,recency,gross_revenue,frequency
count,406516.00,406516.00,406516.00,406516.00
mean,724.68,39.35,11076.01,23.24
std,1491.50,64.49,30154.67,45.51
min,-168469.60,0.00,0.00,1.00
25%,242.26,4.00,1089.18,4.00
50%,386.31,15.00,2618.23,8.00
75%,682.17,40.00,6180.49,18.00
max,168469.60,373.00,279489.02,248.00


### Categorical Attibutes

In [152]:
# Tipos de Registros compostos apenas por strings que aparecem no codigo de estoque 
#(como os códigos são mistos, o Meigarom procurou códigos formados apenas por strings pra encontrar sujeiras
#df5.loc[df5['stock_code'].apply (lambda x:bool (re.search('^[a-zA-Z]+$', x ) ) ), 'stock_code' ].unique()


# <font color='purple'>Filtragem de Variáveis

In [ ]:
df2 = df1.copy() 

In [ ]:
df2.head()

# <font color='red'>EDA(Exploratory Data Analysis)

## Univariate Analysis

## Bivariate Analysis

## Space study

### PCA

### UMAP

### t-SNE

### Tree-Based Embedding

# <font color='red'>Data Preparation

# <font color='red'>Feature Selection

# <font color='red'>Hyperparameter Fine-Tunning

## K-Means

## GMM

## Hierarchical Clustering

## DBSCAN

## Results

## Within-Cluster Sum of Square (WSS)

## Silhouette Score

# <font color='red'>Model Training

## K-Means

## Cluster Validation

# <font color='red'>Cluster Analysis

# <font color='red'>Deploy to production